Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['talk.politics.misc',
              'sci.space']

data = fetch_20newsgroups(subset='train', categories=categories)

In [3]:
fetch_20newsgroups(subset="train").target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
#print out data sample
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
data["target"].shape

(1058,)

In [6]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [7]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [8]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   19.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [9]:
grid_search.best_score_

0.9508316194223376

In [10]:
grid_search.predict_proba(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([[0.61560092, 0.38439908],
       [0.61560092, 0.38439908]])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [11]:
import pandas as pd

df = pd.read_csv("data/train.csv")
df.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [12]:
!pip install 

from FiveOneOne import nlp_utils

def preprocess(doc):
    return " ".join(nlp_utils.simple_tokenize(doc))

df["processed_desc"] = df["description"].apply(preprocess)
df.head()

ERROR: You must give at least one requirement to install (see "pip help install")


,id,description,ratingCategory,processed_desc
0,1321,"\nSometimes, when whisky is batched, a few lef...",1,sometimes whisky batched leftover barrels retu...
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,uncommon exclusive bottling 6 year old cask st...
2,655,\nThis release is a port version of Amrut’s In...,1,release port version amruts intermediate sherr...
3,555,\nThis 41 year old single cask was aged in a s...,1,41 year old single cask aged sherry butt inter...
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1,herbal nose aromas dried tarragon parsley dill...


In [13]:
vect = TfidfVectorizer(stop_words="english", preprocessor=preprocess)
rfc = RandomForestClassifier()

pipe = Pipeline([
    ("vect", vect),
    ("rfc", rfc),
])

from sklearn.model_selection import RandomizedSearchCV
import scipy

search = RandomizedSearchCV(
    pipe,
    param_distributions={
        "vect__max_df": scipy.stats.uniform(0.75, 0.99),
        "vect__min_df": scipy.stats.uniform(0.01, 0.25),
        "vect__max_features": scipy.stats.randint(500, 1000),
        "rfc__n_estimators": scipy.stats.randint(20, 120),
        "rfc__max_depth": scipy.stats.randint(20, 100),
    },
    cv=5,
    n_iter=50,
    n_jobs=-1,
    verbose=1,
)

search.fit(df["description"], df["ratingCategory"])

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.8min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [14]:
search.best_score_

0.7281637453501839

In [15]:
test_df = pd.read_csv("data/test.csv")

test_df["ratingCategory"] = search.predict(test_df["description"])
test_df[["id", "ratingCategory"]].to_csv("submission.csv", header=True, index=False)

## Challenge

You're trying to achieve 75% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [16]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [17]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [18]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [19]:
print(pipe)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 1), norm='l2',
                                                  preprocessor=<function preprocess at 0x7fe2c16fd268>,
                                                  smooth_idf=True,
                                                  sto...
                 RandomForestClassifi

In [20]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(df["description"], df["ratingCategory"])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.9s
/home/ecow/venv/lambda/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [21]:
grid_search.best_score_

0.7191101082438284

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [22]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [23]:
doc = nlp("Two bananas in pyjamas")

In [24]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [25]:
def get_word_vectors(docs):
    return [nlp(preprocess(doc)).vector for doc in docs]

In [26]:
X = get_word_vectors(df['description'])

len(X) == len(data.data)

False

In [27]:
X_test = get_word_vectors(test_df['description'])

In [28]:
search = RandomizedSearchCV(
    rfc,
    param_distributions={
        "n_estimators": scipy.stats.randint(20, 120),
        "max_depth": scipy.stats.randint(10, 25),
        "min_samples_leaf": scipy.stats.randint(1, 5),
    },
    n_jobs=-1,
    cv=5,
    n_iter=25,
    verbose=1
)
search.fit(X, df["ratingCategory"])

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   51.7s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [29]:
#rfc.fit(X, df['ratingCategory'])

In [30]:
search.score(X, df['ratingCategory'])

0.9689258624908246

In [31]:
search.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0])

In [32]:
test_df['ratingCategory'] = search.predict(X_test)

In [33]:
test_df[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

In [34]:
from multiprocessing import Pool, cpu_count
pool = Pool(cpu_count())
x = pool.map(preprocess, df["description"])

## Follow Along

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle competition
* Find another text classification task to work on

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=tYkaAlJNfhul

In [3]:
import sys

sys.path.append("/media/shared/projects/lambda/bert_re")

import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

In [1]:
import pandas as pd

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [2]:
X_train = train_df[["description"]]
y_train = train_df["ratingCategory"]

In [6]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

### now let's load the tokenizer and play with it

In [7]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [15]:
TASK = 'MRPC'

TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = X_train
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
#tpu_cluster_resolver = tf.compat.v1.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.compat.v1.contrib.tpu.RunConfig(
    #cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.compat.v1.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


In [ ]:

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

In [3]:
from simpletransformers.classification import ClassificationModel

train = train_df[["description", "ratingCategory"]]

# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=3, use_cuda=False, args={
    "max_seq_length": 512,
    "learning_rate": 2e-5,
    "warmup_steps": 20,
    
})
model.train_model(train)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/home/ecow/.conda/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/home/ecow/.conda/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [5]:
from joblib import dump

dump(model, "model.joblib")

/home/ecow/.conda/envs/pytorch/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


['model.joblib']

In [6]:
res, outputs, wrong = model.eval_model(train)

/home/ecow/.conda/envs/pytorch/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:744: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


In [7]:
res

{'mcc': 0.44136061911226804, 'eval_loss': 0.5155506254175638}

In [8]:
test = test_df["description"]

pred, raw_out = model.predict(test)

In [9]:
test_df["ratingCategories"] = pred

array([1, 0, 1, ..., 0, 1, 0])

In [10]:
test_df["ratingCategory"] = pred
test_df["ratingCategory"].value_counts()

1    820
0    202
Name: ratingCategory, dtype: int64

In [12]:
test_df[["id", "ratingCategory"]].to_csv("bert_submission.csv", index=False, header=True)

In [37]:
from lightgbm import LGBMClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

svd = TruncatedSVD( # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)
clf = LGBMClassifier()

pipe = Pipeline([
    ("svd", svd),
    ("clf", clf),
])

search = RandomizedSearchCV(
    pipe,
    param_distributions={
        "svd__n_components": scipy.stats.randint(25, 250),
        "clf__num_leaves": scipy.stats.randint(10, 45),
        "clf__max_depth": scipy.stats.randint(15, 45),
        "clf__learning_rate": scipy.stats.uniform(0.05, 0.25),
        "clf__n_estimators": scipy.stats.randint(20, 200),
    },
    cv=5,
    n_iter=100,
    n_jobs=-1,
    verbose=1,
)

y = df["ratingCategory"]

search.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 12.9min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('svd',
                                              TruncatedSVD(algorithm='randomized',
                                                           n_components=2,
                                                           n_iter=10,
                                                           random_state=None,
                                                           tol=0.0)),
                                             ('clf',
                                              LGBMClassifier(boosting_type='gbdt',
                                                             class_weight=None,
                                                             colsample_bytree=1.0,
                                                             importance_type='split',
                                                             learning_rate=0.1,
             

In [38]:
search.best_params_

{'clf__learning_rate': 0.1662097455100487,
 'clf__max_depth': 32,
 'clf__n_estimators': 158,
 'clf__num_leaves': 28,
 'svd__n_components': 135}

In [39]:
search.best_score_

0.7470051144236323